# MinIOにデータを書き込む

In [5]:
from pyspark.sql.functions import col, rand

MinIOはS3互換である。エンドポイント、ID、シークレットキーを指定しながらS3プロトコルを用いて読み書きできる。
なお、あわせて `fs.s3a.path.style.access` を有効化することで名前解決のエラーを避ける。

In [16]:
spark._jsc.hadoopConfiguration().set('fs.s3a.endpoint','http://localhost:9000')
spark._jsc.hadoopConfiguration().set('fs.s3a.access.key','minioadmin')
spark._jsc.hadoopConfiguration().set('fs.s3a.secret.key','minioadmin')
spark._jsc.hadoopConfiguration().set('fs.s3a.path.style.access','true')

ダミーデータを作る。

In [17]:
df01 = spark.range(10).alias('id')
df01.show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



In [18]:
df02 = df01.select(col('id'), rand().alias('rand'))
df02.show()

+---+--------------------+
| id|                rand|
+---+--------------------+
|  0|  0.8583754018110523|
|  1| 0.23327449392043575|
|  2| 0.11184375072823527|
|  3|  0.6663237844847486|
|  4|   0.382967832475844|
|  5|  0.5138345675128307|
|  6|  0.5131521151713943|
|  7|0.016830960632880698|
|  8|  0.4528880979467512|
|  9|  0.9530944735613183|
+---+--------------------+



MinIOに書き込み

In [19]:
output_url = 's3a://test/sample_data'

In [21]:
df02.write.format('delta').mode('overwrite').save(output_url)